![Banner logo](https://raw.githubusercontent.com/CitrineInformatics/community-tools/master/templates/fig/citrine_banner_2.png "Banner logo")

## Exercises in Machine Learning

*Authors: Zach del Rosario (zdelrosario@citrine.io)*

The _primary_ purpose of this notebook is to help you *not get fooled by machine learning*! As Drew Conway notes, possessing hacking skills and substantive experience -- but having no math or statistics background -- puts one in the [danger zone](http://drewconway.com/zia/2013/3/26/the-data-science-venn-diagram). While we can't possibly cover *everything* you need in a single workshop, this exercise will highlight some of the challenges of doing machine learning well. 


### Learning outcomes
By working through this notebook, you will be able to:

* Use scikit-learn to fit regression models to data
* Use cross-validation to help avoid *underfitting* and *overfitting* of data
* Train a machine learning model to predict the fatigue strength of metal alloys

Tips:

* This exercise will make heavy use of [scikit-learn](https://scikit-learn.org/stable/); you can find lots of useful info on the [documentation site](https://scikit-learn.org/stable/documentation.html).

In [ ]:
# Setup
import numpy as np
import pandas as pd

# Model training tools
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_validate
from sklearn.metrics.scorer import make_scorer

# For jupyter-matplotlib compatibility
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

# Custom tools
from workshop_utils import formulas2df

# Agrawal data from previous exercise
filename_data = "./agrawal_data.csv"

# Helper functions
def nde(y_true, y_pred):
    """Non-dimensional Model Error
    """
    mse = mean_squared_error(y_true, y_pred)
    response_std = np.std(y_true)
    
    return np.sqrt(mse) / response_std

nde_score = make_scorer(nde)

def add_noise(function, sigma = 0.1, seed = 101):
    """Add noise to deterministic functions
    """
    def new_function(x):
        np.random.seed(seed)
        y = function(x)
        return y + sigma * np.random.normal(size = y.shape)
    
    return new_function

# Reference points for regression examples
X_ref = np.atleast_2d(np.linspace(-1, +1, num = 100)).T
# Reference models
fcn_1 = lambda x: (0.3 * x**2 + 1.0 * x + 2).flatten()
fcn_1_noisy = add_noise(fcn_1)
fcn_2 = lambda x: (-2.0 * x**3 + 0.4 * x**2 + 1.0 * x + 2).flatten()
fcn_2_noisy = add_noise(fcn_2, sigma = 0.2)

## Primer: The Key Ideas
First we'll cover some key ideas on simple functions. These are not 'real' data, but the simplicity of the examples will allow us to focus on concepts.

Here I generate some data from a simple polynomial.

In [ ]:
## Ground-truth data; no noise
X_c = np.atleast_2d(np.linspace(-1, +1, num = 10)).T
Y_1 = fcn_1(X_c)

We will fit a simple [linear regression](https://en.wikipedia.org/wiki/Linear_regression) to these data. To do this, we'll use the implementation `LinearRegression()` from scikit-learn. To start, we'll assume that the data were generated from an underlying rule (a _model_) of the form

$$y = m x + b,$$

and attempt to _learn_ the slope $m$ and intercept $b$ by _minimizing_ the difference between the measured values `Y_1` and the predicted values `Y_1_linear_pred`.

### Q1: Fit a linear regression
Use the scikit-learn function `LinearRegression()` to fit a line to the data `X_c, Y_1`.

Note: You will have to [look up](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html) the documentation for `LinearRegression()`!

In [ ]:
## TASK: Fit a line on X_c, Y_1
reg_1_linear = None # TODO: Fit a scikit learn regression with LinearRegression()

## Predict using fitted model
Y_1_ref = fcn_1(X_ref)
Y_1_linear_pred = reg_1_linear.predict(np.atleast_2d(X_ref)) # Predict using model from above

## Compute mean-squared error
nde_1 = nde(Y_1_ref, Y_1_linear_pred)
print("nde = {0:4.3f}".format(nde_1))
print("(Should be nde = 0.158)")

## Plot results
plt.figure()
plt.plot(X_ref, Y_1_ref, 'k--', label = "Truth")
plt.plot(X_c, Y_1, 'k.', label = "Measured")
plt.plot(X_ref, Y_1_linear_pred, 'b-', label = "Model")
plt.legend(loc = 0)
plt.show()

### Model Flexibility and _Underfitting_
Note that we had to _assume_ a model-form in order to do the fitting. From the figure above, we can see that the model is close to the true values, but obviously lacks the curvature of the true data-generating process.

This phenomenon -- failing to capture behavior in the data -- is called _underfitting_. This leads to error in the model, which is quantified above using _non-dimensional error_ (NDE). To reduce this contribution to error, we need to make our model _more flexible_ -- one way to do this is to add additional _features_ for the model to fit. 

### Featurization
In this simple problem, we only have a single input $x$. However, we can generate additional _features_ on which to fit by considering additional _powers_ of $x$. For instance, we could fit a quadratic model

$$y = b x^0 + m_1 x^1 + m_2 x^2.$$

It is a _common misconception_ that linear regression _cannot fit nonlinear models_. Clearly the model above is nonlinear, but it is _linear in the features_ $[1, x^1, x^2]$. By providing the additional quadratic term, we give the model more flexibility to fit patterns in the data. Below, you will compute this quadratic featurization by using the `PolynomialFeatures()` function from scikit-learn.

### Q2: Build polynomial features
Use the scikit-learn function `PolynomialFeatures()` to build a matrix of values $[1, x^1, x^2]$.

Note: You'll probably need to look up the documentation for `PolynomialFeatures()`!

In [ ]:
## TASK: Featurize the data
poly_2d = None  # TODO: Generate a transformation object using PolynomialFeatures()
X_quad = X_c    # TODO: Compute featurization to add quadratic term

print(X_quad)
print("(Should be)\n [Constant,    Linear,     Quadratic]")

Note that this featurization gives rows of $[x^0, x^1, x^2]$, as we discussed above. Now we can use the featurized data to fit a quadratic model, and can use the same transform to evaluate the model on the reference points `X_ref`.

### Q3: Fit a quadratic regression
Use the transform `poly_2d` and featurized data `X_quad` to fit and evaluate a quadratic model.

In [ ]:
## TASK: Use your featurization to fit a new model
Y_1_quad_pred = Y_1_ref # TODO: Compute predictions

## Compute mean-squared error
nde_1_quad = nde(Y_1_ref, Y_1_quad_pred)
print("nde = {0:4.3f}".format(nde_1_quad))
print("(Should be nde = 0.000)")

## Plot results
plt.figure()
plt.plot(X_ref, Y_1_ref, 'k--', label = "Truth")
plt.plot(X_c, Y_1, 'k.', label = "Measured")
plt.plot(X_ref, Y_1_quad_pred, 'b-', label = "Model")
plt.legend(loc = 0)
plt.show()

Here we can see the model fits the data _perfectly_, which is corroborated by `nde = 0.000`. This _suggests_ that we have successfully discovered the _exact_ rule that generated these data, which in _this special case happens to be true_.

However, we will very rarely be able to fit the true function exactly. This is because real data tend to have _noise_, which corrupts the underlying function we are trying to learn.

### Noise and _Overfitting_
Below, I generate data from the same model, but add a little bit of noise.

In [ ]:
## Noisy data
Y_1_noisy = fcn_1_noisy(X_c)

First, try fitting a quadratic to these new data, and inspect the fit.

### Q4: Fit a quadratic model to noisy data

In [ ]:
## TASK: Fit a model on X_1, Y_1_noisy
Y_1_noisy_pred = Y_1_ref # TODO: Compute predictions

## Compute mean-squared error
nde_1_noisy = nde(Y_1_ref, Y_1_noisy_pred)
print("nde = {0:4.3f}".format(nde_1_noisy))
print("(Should be nde = 0.139)")

## Plot results
plt.figure()
plt.plot(X_ref, Y_1_ref, 'k--', label = "True")
plt.plot(X_c, Y_1_noisy, 'k.', label = "Measured")
plt.plot(X_ref, Y_1_noisy_pred, 'b-', label = "Model")
plt.legend(loc = 0)
plt.show()

Here we can see that the fit is no longer perfect, despite coming from the "same" model. This is also reflected in the finite NDE value. Since we already know that a quadratic can fit the underlying function perfectly, underfitting is not the issue here. Instead, the error is increased due to the noise in the data.

_However_, we have not yet seen a case of _overfitting_. To see that phenomenon, let's consider a slightly more complicated function.

In [ ]:
# Generate data from the true function
Y_2_ref = fcn_2(X_ref)
# Generate noisy data
Y_2_noisy = fcn_2_noisy(X_c)

plt.figure()
plt.plot(X_ref, Y_2_ref, 'k--', label = "True")
plt.plot(X_c, Y_2_noisy, 'k.', label = "Measured")
plt.legend(loc = 0)
plt.show()

Here we can see a somewhat complicated function that is quite corrupted by noise. Below, I'm going to fit a number of polynomial models of different orders. In practice, we would like to _make a decision_ about what polynomial order to use. A sensible choice would be to pick the order that minimizes the error -- let's see which model accomplishes this.

In [ ]:
# Fit and predict on same data
def fit_poly(X, Y, order):
    poly = PolynomialFeatures(order)
    X_poly = poly.fit_transform(X)
    
    reg = LinearRegression().fit(X_poly, Y)
    
    return reg, poly

Ord_all = list(range(15))
Ord_plot = [1, 3, 9]
Y_pred_c = np.zeros((len(Ord_all), X_c.shape[0]))
Y_pred_ref = np.zeros((len(Ord_all), X_ref.shape[0]))
NDE_c = np.zeros(len(Ord_all))
NDE_ref = np.zeros(len(Ord_all))

plt.figure()
plt.plot(X_ref, Y_2_ref, 'k--')
for i in range(len(Ord_all)):
    # Fit model
    reg, poly = fit_poly(X_c, Y_2_noisy, Ord_all[i])
    # Predict on same data & on reference points
    Y_pred_c[i] = reg.predict(poly.fit_transform(X_c))
    Y_pred_ref[i] = reg.predict(poly.fit_transform(X_ref))
    # Compute error *on same data* -> estimated error
    NDE_c[i] = nde(Y_2_noisy, Y_pred_c[i])
    # Compute error on reference points -> 'true' error
    NDE_ref[i] = nde(Y_2_ref, Y_pred_ref[i])
    
    # Plot curve
    if Ord_all[i] in Ord_plot:
        plt.plot(X_ref, Y_pred_ref[i], label = "Order = {}".format(Ord_all[i]))
plt.plot(X_c, Y_2_noisy, 'k.')
plt.legend(loc = 0)
plt.show()    

Here I've selected just a few of the models to plot. We can see

* The `Order = 1` case is underfit, like we saw in the example above
* The `Order = 9` case curves tortuously to go through _every single point_; this is an example of _overfitting_
* The `Order = 3` case is not perfect, but fairly close to the true (dashed) curve. This is a well-fit model.

More generally, _overfitting_ is when the model fits to spurrious patterns in the data; essentially, we are fitting to noise, rather than signal. We would like to detect and avoid overfitting in practice! While we can see above some suspicious behavior based on the fitted curves, we might like a _quantitative_ way to compare models. We can do this with the NDE values, but there is a _subtle issue_ at play here.

Let's compare the `NDE` values `Estimated` on only the available (noisy) data, and the error computed using evaluations from the `True` (noiseless) function.

In [ ]:
plt.figure()
plt.plot(Ord_all, NDE_c, label = 'Estimated')
plt.plot(Ord_all, NDE_ref, label = 'True')
plt.legend(loc = 0)
plt.xlabel('Poly Order')
plt.ylabel('ND Error')
plt.show()

Here we can see the `Estimated` and `True` error values _greatly diverge_. This is _highly problematic_ for two interrelated reasons:

1. In practice, we would only have access to the `Estimated` curve, as the `True` curve relies on data we do not have.
2. If we were to make a decision about `Poly Order` based on the `Estimated` curve, we would choose a much higher order than what would minimize the NDE in the `True` case.

The underlying reason for the poor error estimate here is that _we are using the same data to both train and test the model_. We can improve our estimates for the error through various techniques; below, we will use the technique of _cross-validation_.

### Avoiding Optimistic Estimates: Cross-Validation
[Cross-validation](https://en.wikipedia.org/wiki/Cross-validation_(statistics)) is a technique for estimating the error in a way that avoids the "optimism" we saw above. For the variant _k-fold cross-validation_, we split all our data into _folds_, and use these to build _training_ and _test_ sets. Generally:

* _Training_ data are used to fit a model
* _Test_ data are used to evaluate a model

![CV schematic](https://upload.wikimedia.org/wikipedia/commons/1/1c/K-fold_cross_validation_EN.jpg)

(Fabian Flock, via Wikimedia)

In each of our $k$ iterations, we _do not allow_ the model to see a test fold, and fit the model only on the remaining data. We then compute our chosen error metric on the test fold. This gives us a less optimistic estimate for the error.

This procedure is implemented in the scikit-learn function `cross_validate()`. Use this routine to estimate the NDE in the case where the polynomial order is `9`. You will have to choose a number of folds to run; while `5` to `10` is common, since we have so few data, you will need to use a smaller number of folds.

### Q5: K-fold Cross-Validation
Perform k-fold cross validation on the order `9` polynomial model using the scikit-learn function `cross_validate()`.

In [ ]:
## TASK: Use cross_validate() on the 9th order model to estimate the NDE
## Hint, you can pass the helper function `nde_score` (defined above) to cross_validate()

scores

The `train_score` values are quite optimistic, while the `test_score` values are *abysmal*.

Below, I show results for performing k-fold cross validation across the same set of polynomial orders as above.

In [ ]:
# REDACTED....


Here we can see that the NDE takes reasonable values for order at or below `3`. Beyond this point, the NDE explodes as models begin to overfit wildly. These cross-validated error metrics would be far more informative for making a decision about polynomial order.

Here we have just one tunable knob (polynomial order) that defines our model. More generally, these kinds of user-selected quantities are called [hyperparameters](https://en.wikipedia.org/wiki/Hyperparameter_(machine_learning)). Cross-validation and related techniques are key to _tuning hyperparameters_.

### Using Models for Materials Informatics
In the final part of the workshop, we will discuss how to _use_ these machine learning models to do useful work in materials science. Stay tuned!